In [1]:
from datetime import datetime
from influxdb import DataFrameClient
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName("Test").getOrCreate()
df = pd.read_excel('SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0'])

df.head()
#sdf = spark.createDataFrame(df) sdf.show()

,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
0,2019-07-20 08:39:59.004013+00:00,4.323736,492.896881,2,2,2,1,131.408615,9.313829,257.933868,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
1,2019-07-20 08:39:58.005004800+00:00,4.323736,492.465100,2,2,2,1,131.408615,9.316713,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
2,2019-07-20 08:39:57.005004800+00:00,4.303558,492.308100,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
3,2019-07-20 08:39:56.005004800+00:00,4.253915,491.405273,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
4,2019-07-20 08:39:55.001007+00:00,4.200429,491.169769,2,2,2,1,131.408615,9.319918,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


In [6]:
pip install rbf

In [ ]:
import numpy as np
from rbf import rbf
filepath ='SWaT_dataset_Jul 19 v2.xlsx'
matrixX =df.drop('GMT +0',axis='columns')

# form the RBF kernel matrix
matrixX1 = matrixX[0:200, :]
matrixX2 = matrixX[200:700, :]
sigma = 0.7
matrixA = rbf(matrixX1, matrixX2, sigma)
print('The matrix A has ', matrixA.shape[0], ' rows and ', matrixA.shape[1], ' columns.')

In [ ]:
import numpy as np

def adaptiveSampling(matrixA, matrixC1, c2):
    # compute the residual
    matrixQ, matrixR = np.linalg.qr(matrixC1, mode='reduced')
    matrixQQA = np.dot(matrixQ, np.dot(matrixQ.transpose(), matrixA))
    matrixRes = matrixA - matrixQQA
    # compute the sampling probabilites
    matrixRes = np.square(matrixRes)
    prob = sum(matrixRes)
    prob = prob / sum(prob)
    return np.random.choice(matrixA.shape[1], c2, replace=False, p=prob)

c1 = 100;
c2 = 200;
# matrix C1 contains c1 uniformly sampled columns of matrix A
indexC1 = np.random.choice(matrixA.shape[1], c1, replace=False)
matrixC1 = matrixA[:, indexC1]
indexC2 = adaptiveSampling(matrixA, matrixC1, c2)
matrixC2 = matrixA[:, indexC2]
matrixC = np.concatenate((matrixC1, matrixC2), 1)

In [ ]:
matrixQ, matrixR = np.linalg.qr(matrixC, mode='reduced')
matrixRes = matrixA - np.dot(matrixQ, np.dot(matrixQ.T, matrixA))
approxError = np.linalg.norm(matrixRes, 'fro') / np.linalg.norm(matrixA, 'fro')
print(approxError)
del(matrixR, matrixQ, matrixRes, approxError)